# Домашнее задание. Часть 2 (HW)

Задача

1. Обучите несколько моделей рекуррентных нейронных сетей, например LSTM, GRU, Bidirectional-LSTM.
   
2. Посчитайте значение метрики, которую вы предложили в части №1 и сравните результаты для разных RNN, эвристик и классического ML.

## Обучение моделей RNN

### LTSM 

In [73]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import pandas as pd


df = pd.read_json(r'data\data.json')

df['label'] = df['sentiment'].map({'extremely negative': 0, 'negative': 1, 'neutral':2, 'positive':3, "extremely positive":4})


X_train, X_test, y_train, y_test = train_test_split(df['lemmatization'], df["label"], test_size=0.2, random_state=42)

# Параметры
max_words = 10000  # Максимальное количество слов в словаре

# Токенизация
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)


max_length = max(len(x) for x in X_train_seq)
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length)

df

,id,text,sentiment,lemmatization,text_length,word_count,stopword_count,unique_word_count,unique_to_total_ratio,polarity,trigrams,positive,negative,weight,classes,label
0,1,advice talk to your neighbours family to excha...,positive,advice talk to your neighbour family to exchan...,228,38,11,33,0.868421,0.166667,"[[advice, talk, to], [talk, to, your], [to, yo...",1,0,1,4,3
1,2,coronavirus australia woolworths to give elder...,positive,coronavirus australia woolworth to give elderl...,102,13,1,13,1.000000,-0.200000,"[[coronavirus, australia, woolworth], [austral...",1,2,-1,2,3
2,3,my food stock is not the only one which is emp...,positive,my food stock is not the only one which is emp...,253,41,18,36,0.878049,0.200000,"[[my, food, stock], [food, stock, is], [stock,...",3,1,2,4,3
3,4,me ready to go at supermarket during the covid...,extremely negative,me ready to go at supermarket during the covid...,257,39,16,35,0.897436,-0.077778,"[[me, ready, to], [ready, to, go], [to, go, at...",1,4,-3,1,0
4,5,as news of the regionas first confirmed covid1...,positive,a news of the regionas first confirmed covid19...,212,35,9,33,0.942857,0.198611,"[[a, news, of], [news, of, the], [of, the, reg...",1,0,1,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41151,41152,airline pilots offering to stock supermarket s...,neutral,airline pilot offering to stock supermarket sh...,72,11,2,11,1.000000,0.000000,"[[airline, pilot, offering], [pilot, offering,...",0,0,0,3,2
41152,41153,response to complaint not provided citing covi...,extremely negative,response to complaint not provided citing covi...,132,22,7,21,0.954545,-0.150000,"[[response, to, complaint], [to, complaint, no...",1,3,-2,2,0
41153,41154,you know itas getting tough when kameronwilds ...,positive,you know itas getting tough when kameronwilds ...,126,18,4,18,1.000000,-0.211111,"[[you, know, itas], [know, itas, getting], [it...",1,0,1,4,3
41154,41155,is it wrong that the smell of hand sanitizer i...,neutral,is it wrong that the smell of hand sanitizer i...,102,18,9,16,0.888889,-0.250000,"[[is, it, wrong], [it, wrong, that], [wrong, t...",0,2,-2,2,2


In [72]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Параметры
embedding_dim = 128  # Размерность эмбеддингов
lstm_units = 128     # Количество нейронов в слое LSTM

# Создание модели
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_length))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))

optimizer = Adam(learning_rate=0.0001)

# Компиляция модели
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Вывод структуры модели
model.summary()

d:\URFU\III MOMO\HW1\.venv\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [71]:
# Обучение
history = model.fit(X_train_pad, y_train,
                    epochs=10,   # Количество эпох
                    batch_size=32,  # Размер батча
                    validation_split=0.2)


loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")



Epoch 1/10
824/824 ━━━━━━━━━━━━━━━━━━━━ 28s 34ms/step - accuracy: 0.0602 - loss: 0.0000e+00 - val_accuracy: 0.0603 - val_loss: 0.0000e+00
Epoch 2/10
824/824 ━━━━━━━━━━━━━━━━━━━━ 29s 35ms/step - accuracy: 0.0650 - loss: 0.0000e+00 - val_accuracy: 0.0603 - val_loss: 0.0000e+00
Epoch 3/10
824/824 ━━━━━━━━━━━━━━━━━━━━ 29s 35ms/step - accuracy: 0.0652 - loss: 0.0000e+00 - val_accuracy: 0.0603 - val_loss: 0.0000e+00
Epoch 4/10
824/824 ━━━━━━━━━━━━━━━━━━━━ 29s 35ms/step - accuracy: 0.0623 - loss: 0.0000e+00 - val_accuracy: 0.0603 - val_loss: 0.0000e+00
Epoch 5/10
457/824 ━━━━━━━━━━━━━━━━━━━━ 12s 35ms/step - accuracy: 0.0613 - loss: 0.0000e+00

KeyboardInterrupt: 

In [ ]:


# Новый текст
new_texts = ["Этот продукт ужасен!", "Очень доволен покупкой!"]

# Преобразование текста в числовые последовательности
new_sequences = tokenizer.texts_to_sequences(new_texts)
new_X = pad_sequences(new_sequences, maxlen=max_len)

# Предсказания
predictions = model.predict(new_X)
print(predictions)  # Значения от 0 до 1

# Интерпретация результатов
predicted_labels = [1 if p > 0.5 else 0 for p in predictions]
print(predicted_labels)  # [0, 1] (негативный, позитивный)

In [ ]:
import numpy as np

# Загрузка GloVe
embedding_index = {}
with open("glove.6B.100d.txt", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embedding_index[word] = coefs

# Создание матрицы эмбеддингов
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < max_words:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# Встраивание предобученных эмбеддингов в модель
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len, weights=[embedding_matrix], trainable=False))
model.add(LSTM(lstm_units, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))